# Use DDPG to Play Pendulum-v0

PyTorch version

In [1]:
%matplotlib inline

import sys
import logging
import imp
import itertools
import copy

import numpy as np
np.random.seed(0)
import pandas as pd
import gym
import matplotlib.pyplot as plt
import torch
torch.manual_seed(0)
import torch.nn as nn
import torch.optim as optim

imp.reload(logging)
logging.basicConfig(level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s',
        stream=sys.stdout, datefmt='%H:%M:%S')

In [2]:
env = gym.make('Pendulum-v0')
env.seed(0)
for key in vars(env):
    logging.info('%s: %s', key, vars(env)[key])

21:21:54 [INFO] env: <PendulumEnv<Pendulum-v0>>
21:21:54 [INFO] action_space: Box(-2.0, 2.0, (1,), float32)
21:21:54 [INFO] observation_space: Box(-8.0, 8.0, (3,), float32)
21:21:54 [INFO] reward_range: (-inf, inf)
21:21:54 [INFO] metadata: {'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': 30}
21:21:54 [INFO] _max_episode_steps: 200
21:21:54 [INFO] _elapsed_steps: None


In [3]:
class DQNReplayer:
    def __init__(self, capacity):
        self.memory = pd.DataFrame(index=range(capacity),
                columns=['observation', 'action', 'reward',
                'next_observation', 'done'])
        self.i = 0
        self.count = 0
        self.capacity = capacity
    
    def store(self, *args):
        self.memory.loc[self.i] = args
        self.i = (self.i + 1) % self.capacity
        self.count = min(self.count + 1, self.capacity)
        
    def sample(self, size):
        indices = np.random.choice(self.count, size=size)
        return (np.stack(self.memory.loc[indices, field]) for field in
                self.memory.columns)

In [4]:
class OrnsteinUhlenbeckProcess:
    def __init__(self, x0):
        self.x = x0

    def __call__(self, mu=0., sigma=1., theta=.15, dt=.01):
        n = np.random.normal(size=self.x.shape)
        self.x += (theta * (mu - self.x) * dt + sigma * np.sqrt(dt) * n)
        return self.x

In [5]:
class DDPGAgent:
    def __init__(self, env):
        state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.shape[0]
        self.action_low = env.action_space.low[0]
        self.action_high = env.action_space.high[0]
        self.gamma = 0.99

        self.replayer = DQNReplayer(20000)

        self.actor_evaluate_net = self.build_net(
                input_size=state_dim, hidden_sizes=[32, 64],
                output_size=self.action_dim) ## WARN: No-OA
        self.actor_optimizer = optim.Adam(self.actor_evaluate_net.parameters(), lr=0.0001)
        self.actor_target_net = copy.deepcopy(self.actor_evaluate_net)

        self.critic_evaluate_net = self.build_net(
                input_size=state_dim+self.action_dim, hidden_sizes=[64, 128])
        self.critic_optimizer = optim.Adam(self.critic_evaluate_net.parameters(), lr=0.001)
        self.critic_loss = nn.MSELoss()
        self.critic_target_net = copy.deepcopy(self.critic_evaluate_net)
        
    def build_net(self, input_size, hidden_sizes, output_size=1,
            output_activator=None):
        layers = []
        for input_size, output_size in zip(
                [input_size,] + hidden_sizes, hidden_sizes + [output_size,]):
            layers.append(nn.Linear(input_size, output_size))
            layers.append(nn.ReLU())
        layers = layers[:-1]
        if output_activator:
            layers.append(output_activator)
        net = nn.Sequential(*layers)
        return net

    def reset(self, mode=None):
        self.mode = mode
        if self.mode == 'train':
            self.trajectory = []
            self.noise = OrnsteinUhlenbeckProcess(np.zeros((self.action_dim,)))

    def step(self, observation, reward, done):
        if self.mode == 'train' and self.replayer.count < 3000:
            action = np.random.uniform(self.action_low, self.action_high)
        else:
            state_tensor = torch.as_tensor(observation, dtype=torch.float).reshape(1, -1)
            action_tensor = self.actor_evaluate_net(state_tensor)
            action = action_tensor.detach().numpy()[0]
        if self.mode == 'train':
            noise = self.noise(sigma=0.1)
            action = (action + noise).clip(self.action_low, self.action_high)

            self.trajectory += [observation, reward, done, action]
            if len(self.trajectory) >= 8:
                state, _, _, action, next_state, reward, done, _ = self.trajectory[-8:]
                self.replayer.store(state, action, reward, next_state, done)

            if self.replayer.count >= 3000:
                self.learn()
        return action

    def close(self):
        pass

    def update_net(self, target_net, evaluate_net, learning_rate=0.005):
        for target_param, evaluate_param in zip(
                target_net.parameters(), evaluate_net.parameters()):
            target_param.data.copy_(learning_rate * evaluate_param.data
                    + (1 - learning_rate) * target_param.data)

    def learn(self):
        # replay
        states, actions, rewards, next_states, dones = self.replayer.sample(64)
        state_tensor = torch.as_tensor(states, dtype=torch.float)
        action_tensor = torch.as_tensor(actions, dtype=torch.long)
        reward_tensor = torch.as_tensor(rewards, dtype=torch.float)
        next_state_tensor = torch.as_tensor(next_states, dtype=torch.float)
        done_tensor = torch.as_tensor(dones, dtype=torch.float)
        
        # learn critic
        next_action_tensor = self.actor_target_net(next_state_tensor)
        noise_tensor = (0.2 * torch.randn_like(action_tensor, dtype=torch.float))
        noisy_next_action_tensor = (next_action_tensor + noise_tensor).clamp(
                self.action_low, self.action_high)
        next_state_action_tensor = torch.cat([next_state_tensor, noisy_next_action_tensor], 1)
        next_q_tensor = self.critic_target_net(next_state_action_tensor).squeeze(1)
        critic_target_tensor = reward_tensor + (1. - done_tensor) * self.gamma * next_q_tensor
        critic_target_tensor = critic_target_tensor.detach()
        
        state_action_tensor = torch.cat([state_tensor, action_tensor], 1)
        critic_pred_tensor = self.critic_evaluate_net(state_action_tensor).squeeze(1)
        critic_loss_tensor = self.critic_loss(critic_pred_tensor, critic_target_tensor)
        self.critic_optimizer.zero_grad()
        critic_loss_tensor.backward()
        self.critic_optimizer.step()

        # learn actor
        pred_action_tensor = self.actor_evaluate_net(state_tensor)
        pred_action_tensor = pred_action_tensor.clamp(self.action_low, self.action_high)
        pred_state_action_tensor = torch.cat([state_tensor, pred_action_tensor], 1)
        critic_pred_tensor = self.critic_evaluate_net(pred_state_action_tensor)
        actor_loss_tensor = -critic_pred_tensor.mean()
        self.actor_optimizer.zero_grad()
        actor_loss_tensor.backward()
        self.actor_optimizer.step()

        self.update_net(self.critic_target_net, self.critic_evaluate_net)
        self.update_net(self.actor_target_net, self.actor_evaluate_net)


agent = DDPGAgent(env)

21:21:54 [INFO] ==== train ====
21:21:55 [DEBUG] train episode 0: reward = -1779.61, steps = 200
21:21:55 [DEBUG] train episode 1: reward = -1026.05, steps = 200
21:21:55 [DEBUG] train episode 2: reward = -1556.83, steps = 200
21:21:55 [DEBUG] train episode 3: reward = -1127.41, steps = 200
21:21:55 [DEBUG] train episode 4: reward = -1675.42, steps = 200
21:21:55 [DEBUG] train episode 5: reward = -1632.59, steps = 200
21:21:55 [DEBUG] train episode 6: reward = -750.72, steps = 200
21:21:55 [DEBUG] train episode 7: reward = -1794.28, steps = 200
21:21:55 [DEBUG] train episode 8: reward = -928.49, steps = 200
21:21:56 [DEBUG] train episode 9: reward = -1680.41, steps = 200
21:21:56 [DEBUG] train episode 10: reward = -1312.04, steps = 200
21:21:56 [DEBUG] train episode 11: reward = -873.37, steps = 200
21:21:56 [DEBUG] train episode 12: reward = -1509.77, steps = 200
21:21:56 [DEBUG] train episode 13: reward = -994.03, steps = 200
21:21:56 [DEBUG] train episode 14: reward = -909.50, steps

21:54:44 [DEBUG] test episode 60: reward = -128.80, steps = 200
21:54:48 [DEBUG] test episode 61: reward = -131.01, steps = 200
21:54:51 [DEBUG] test episode 62: reward = -243.42, steps = 200
21:54:55 [DEBUG] test episode 63: reward = -4.31, steps = 200
21:54:58 [DEBUG] test episode 64: reward = -259.29, steps = 200
21:55:01 [DEBUG] test episode 65: reward = -132.05, steps = 200
21:55:05 [DEBUG] test episode 66: reward = -127.97, steps = 200
21:55:08 [DEBUG] test episode 67: reward = -329.27, steps = 200
21:55:11 [DEBUG] test episode 68: reward = -338.29, steps = 200
21:55:15 [DEBUG] test episode 69: reward = -120.85, steps = 200
21:55:18 [DEBUG] test episode 70: reward = -238.73, steps = 200
21:55:21 [DEBUG] test episode 71: reward = -127.81, steps = 200
21:55:25 [DEBUG] test episode 72: reward = -252.92, steps = 200
21:55:28 [DEBUG] test episode 73: reward = -122.29, steps = 200
21:55:32 [DEBUG] test episode 74: reward = -246.82, steps = 200
21:55:35 [DEBUG] test episode 75: reward =

In [6]:
def play_episode(env, agent, max_episode_steps=None, mode=None, render=False):
    observation, reward, done = env.reset(), 0., False
    agent.reset(mode=mode)
    episode_reward, elapsed_steps = 0., 0
    while True:
        action = agent.step(observation, reward, done)
        if render:
            env.render()
        if done:
            break
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        elapsed_steps += 1
        if max_episode_steps and elapsed_steps >= max_episode_steps:
            break
    agent.close()
    return episode_reward, elapsed_steps


logging.info('==== train ====')
episode_rewards = []
for episode in itertools.count():
    episode_reward, elapsed_steps = play_episode(env.unwrapped, agent,
            max_episode_steps=env._max_episode_steps, mode='train')
    episode_rewards.append(episode_reward)
    logging.debug('train episode %d: reward = %.2f, steps = %d',
            episode, episode_reward, elapsed_steps)
    if np.mean(episode_rewards[-10:]) > -120:
        break
plt.plot(episode_rewards)


logging.info('==== test ====')
episode_rewards = []
for episode in range(100):
    episode_reward, elapsed_steps = play_episode(env, agent)
    episode_rewards.append(episode_reward)
    logging.debug('test episode %d: reward = %.2f, steps = %d',
            episode, episode_reward, elapsed_steps)
logging.info('average episode reward = %.2f ± %.2f',
        np.mean(episode_rewards), np.std(episode_rewards))

In [7]:
env.close()